# Analysis of the Combined data for 7pad Scanning

In [2]:
from single_pad_analysis.analysis_functions import *
from single_pad_analysis.HexDetector import HexDetector

In [3]:
run_number = 325
basedir_path = '/home/akallits/Documents/PicoAnalysis/Saclay_Analysis/data/2023_April_h4/processedTrees/ParameterTrees/'
get_run_file_path = lambda pool_num: f'{basedir_path}Run{run_number}-Pool{pool_num}_treeParam.root'
pool_number_dict = { #key:value
    2: 0,
    3: 0,
    4: 0,
    5: 0 }

In [ ]:
for pool_number in pool_number_dict.keys():
    file_path = get_run_file_path(pool_number)
    tree = get_tree(file_path, 'ParameterTree')


